In [1]:
import numpy as np # linear algebra
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from kan import *

In [3]:
df = pd.read_csv("delays.csv")
df.head()

,Previous_Arrival,Previous_Delay,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,SCHEDULED_DEPARTURE,SCHEDULED_TIME,DISTANCE,SCHEDULED_ARRIVAL,ARRIVAL_DELAY,Ground_Time
0,0.0,0.0,2015,2,4,3,UA,548.0,172.0,1208,780.0,-5.0,0.0
1,780.0,-5.0,2015,3,12,4,UA,596.0,94.0,337,630.0,-1.0,1256.0
2,630.0,-1.0,2015,4,30,4,UA,770.0,128.0,787,1018.0,-9.0,140.0
3,1018.0,-9.0,2015,5,8,5,UA,1072.0,168.0,925,1180.0,16.0,54.0
4,1180.0,16.0,2015,7,29,3,UA,640.0,167.0,1005,867.0,-7.0,900.0


In [13]:
df.drop(["AIRLINE"], axis= 1 , inplace= True)

In [15]:
df.head()

,Previous_Arrival,Previous_Delay,YEAR,MONTH,DAY,DAY_OF_WEEK,SCHEDULED_DEPARTURE,SCHEDULED_TIME,DISTANCE,SCHEDULED_ARRIVAL,ARRIVAL_DELAY,Ground_Time
0,0.0,0.0,2015,2,4,3,548.0,172.0,1208,780.0,-5.0,0.0
1,780.0,-5.0,2015,3,12,4,596.0,94.0,337,630.0,-1.0,1256.0
2,630.0,-1.0,2015,4,30,4,770.0,128.0,787,1018.0,-9.0,140.0
3,1018.0,-9.0,2015,5,8,5,1072.0,168.0,925,1180.0,16.0,54.0
4,1180.0,16.0,2015,7,29,3,640.0,167.0,1005,867.0,-7.0,900.0


In [23]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X = df.drop(["ARRIVAL_DELAY"], axis= 1)
y = df["ARRIVAL_DELAY"]
y = y> 15
y = y.replace({True:1,False:0})
y

C:\Users\akiri\AppData\Local\Temp\ipykernel_2476\321698353.py:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y = y.replace({True:1,False:0})


0        0
1        0
2        0
3        1
4        0
        ..
57970    0
57971    1
57972    1
57973    1
57974    0
Name: ARRIVAL_DELAY, Length: 57975, dtype: int64

In [25]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [27]:
import torch.nn as nn
dataset = {}
dtype = torch.get_default_dtype()
dataset['train_input'] = torch.from_numpy(X_train_scaled).type(dtype)
dataset['test_input'] = torch.from_numpy(X_test_scaled).type(dtype)
dataset['train_label'] = torch.from_numpy(y_train.to_numpy()).type(torch.long)
dataset['test_label'] = torch.from_numpy(y_test.to_numpy()).type(torch.long)

In [31]:
model = KAN(width=[11,13,2], grid =4, k = 3, seed=2024)
def train_acc():
    return torch.mean((torch.argmax(model(dataset['train_input']), dim=1) == dataset['train_label']).type(dtype))

def test_acc():
    return torch.mean((torch.argmax(model(dataset['test_input']), dim=1) == dataset['test_label']).type(dtype))
results = model.fit(dataset, opt="LBFGS", steps=20,metrics=(train_acc, test_acc), loss_fn=torch.nn.CrossEntropyLoss());

checkpoint directory created: ./model
saving model version 0.0


| train_loss: 6.49e-01 | test_loss: 6.81e-01 | reg: 5.85e+01 | : 100%|█| 20/20 [09:53<00:00, 29.67s/

saving model version 0.1


In [61]:
print("train_acc")
print(torch.mean((torch.argmax(model(dataset['train_input']), dim=1) == dataset['train_label']).type(dtype)))

train_acc
tensor(0.8268)


In [63]:
print("test_acc")
print(torch.mean((torch.argmax(model(dataset['test_input']), dim=1) == dataset['test_label']).type(dtype)))

test_acc
tensor(0.8202)


In [65]:
unique_values , amount = torch.unique(torch.argmax(model(dataset['train_input']), dim=1) , return_counts=True)
print(f"unique_v: {unique_values}")
print(f"amount: {amount}")

unique_v: tensor([0, 1])
amount: tensor([38387,   456])
